OUR Text Data

In [111]:
import pandas as pd

In [112]:
df  = pd.read_csv('flickr8k/captions.txt')

In [113]:
print(df.size)


80910


In [114]:
print(df.head()) 

                       image  \
0  1000268201_693b08cb0e.jpg   
1  1000268201_693b08cb0e.jpg   
2  1000268201_693b08cb0e.jpg   
3  1000268201_693b08cb0e.jpg   
4  1000268201_693b08cb0e.jpg   

                                             caption  
0  A child in a pink dress is climbing up a set o...  
1              A girl going into a wooden building .  
2   A little girl climbing into a wooden playhouse .  
3  A little girl climbing the stairs to her playh...  
4  A little girl in a pink dress going into a woo...  


In [115]:
#see for each image there are 5 text lines 1->a  1->b  .. 1->e   let group them 
#now can groupy
image_captions = df.groupby("image")["caption"].apply(list).to_dict()
# df.groupby("image") – groups the DataFrame rows by the "image" column
# ["caption"] – selects the "caption" column within each group.
# .apply(list) – converts all captions in each group into a list.
# .to_dict() – converts the grouped result into a dictionary.

In [116]:
first_key = list(image_captions.keys())[0]
print(first_key)
print(image_captions[first_key])
print('10815824_2997e03d76.jpg')
print(image_captions['10815824_2997e03d76.jpg'])

1000268201_693b08cb0e.jpg
['A child in a pink dress is climbing up a set of stairs in an entry way .', 'A girl going into a wooden building .', 'A little girl climbing into a wooden playhouse .', 'A little girl climbing the stairs to her playhouse .', 'A little girl in a pink dress going into a wooden cabin .']
10815824_2997e03d76.jpg
['A blonde horse and a blonde girl in a black sweatshirt are staring at a fire in a barrel .', 'A girl and her horse stand by a fire .', "A girl holding a horse 's lead behind a fire .", 'A man , and girl and two horses are near a contained fire .', 'Two people and two horses watching a fire .']


In [117]:
type(image_captions)

dict

Vocabulary or Keyword Set

In [118]:
from collections import defaultdict
# """
#  Summary
# Feature	dict	defaultdict
# Handles missing keys	❌ No (gives error)	✅ Yes (auto-creates default)
# Needs key check	✅ Yes	❌ No
# Cleaner code	❌ Longer	✅ Shorter
# Use when	You want full control	You want convenience
# """

In [119]:
import nltk
# print(nltk.data.path)


In [120]:
import nltk
# nltk.download('punkt', download_dir='/usr/local/share/nltk_data')
 #provides: A pre-trained tokenizer model that knows how to break English sentences into words and punctuation.
# nltk.download('averaged_perceptron_tagger') # provides: A model that labels each word with its part-of-speech (noun, verb, adjective, etc.)

In [121]:
import nltk
import os

nltk.data.path = [os.path.expanduser("~/nltk_data")]  # ✅ must come before pos_tag


In [122]:
from nltk import word_tokenize, pos_tag



In [123]:
image_keywords = defaultdict(set)  #it is a dict having key as image 1.jpg and value as set of noun words
# Creates an empty defaultdict where each value is a set.
# 💡 Why: We’ll store, for each image, the set of all nouns found in its captions.
# ✨ Benefit: No need to check if the key exists before adding to it.

for image, captions in image_captions.items():
    for caption in captions:
        words = word_tokenize(caption.lower())
        nouns = [word for word, tag in pos_tag(words) if tag.startswith('NN')]
        # nouns = []    or this for loop
        # for word, tag in pos_tag(words):
            #if tag.startswith('NN'):
                #     nouns.append(word)
        image_keywords[image].update(nouns)
   




In [124]:
first_key_image = list(image_captions.keys())[0]
print("Keywords for", first_key_image, ":", image_keywords[first_key_image])


Keywords for 1000268201_693b08cb0e.jpg : {'playhouse', 'girl', 'building', 'cabin', 'dress', 'stairs', 'way', 'entry', 'child', 'set', 'pink'}


Making Triplets


In [125]:
# we do like any got keywords same >0 then simi else disimi
import random # pick a random positive and a random negative example for each anchor.

In [126]:
#now will iterate over all image get its keyworks from image_captions to compare 
triplets = []
image_list = list(image_keywords.keys())
#now iterate and take respective keywords for each image
for anchor in image_list:
    anchor_keywords = image_keywords[anchor]
    
    #now to find positive at least one same keyword
    positives = [img for img in image_list if img!=anchor and  len(image_keywords[img] & anchor_keywords)>0]
    negatives = [img for img in image_list if img!=anchor and len(image_keywords[img] & anchor_keywords)==0]
    
    if positives and negatives:
        pos = random.choice(positives)
        neg = random.choice(negatives)
        triplets.append((anchor,pos,neg))

print("Total triplets created:", len(triplets))
print("Example triplet:", triplets[0])

#see now i caan access keywords also by these keys(image name in triplets)

Total triplets created: 8091
Example triplet: ('1000268201_693b08cb0e.jpg', '3767841911_6678052eb6.jpg', '530454257_66d58b49ee.jpg')


This is imp . as see in above for one anchor 1 triplet , in this if po==5  ne =8 i can make make many diff with unique anchor , more diverse data but size more

In [127]:
# import random

# triplets = []
# image_list = list(image_keywords.keys())

# # Example “combos” of (pos_count, neg_count) you’d like to try:
# combos = [(1, 3),   # pick 1 positive & 3 negatives
#           (5, 6)]   # pick 5 positives & 6 negatives

# for anchor in image_list:
#     anchor_keywords = image_keywords[anchor]
    
#     # Build your full candidate lists once per anchor
#     positives = [
#         img for img in image_list
#         if img != anchor
#            and len(image_keywords[img] & anchor_keywords) > 0
#     ]
#     negatives = [
#         img for img in image_list
#         if len(image_keywords[img] & anchor_keywords) == 0
#     ]
    
#     # Try each (pos_no, neg_no) pair
#     for pos_no, neg_no in combos:
#         # Ensure you actually have enough candidates
#         if pos_no <= len(positives) and neg_no <= len(negatives):
#             # Randomly pick pos_no distinct positives
#             pos_samples = random.sample(positives, pos_no)    
#             # Randomly pick neg_no distinct negatives
#             neg_samples = random.sample(negatives, neg_no)    

#             # Now form every combination of those picks
#             for pos in pos_samples:            # pos is a str (filename)
#                 for neg in neg_samples:        # neg is a str (filename)
#                     triplets.append((anchor, pos, neg))


Now code till training model [img emb , text emb , fuse , get emb fn , triplet loss , training loop]

In [128]:
!pip install tensorflow


In [129]:
import tensorflow as tf


In [130]:
#see need to get models
!pip install tensorflow tensorflow-hub

In [131]:
#model defifnatin

import tensorflow_hub as hub
from tensorflow.keras import layers,Model

# Model
# ├── 📷 Image Branch
# │   ├── ResNet50 (frozen)
# │   │   └── Output: 2048-d
# │   └── img_proj (Dense 256 + ReLU)
# │       └── Output: 256-d
# │
# ├── 📝 Text Branch
# │   ├── Universal Sentence Encoder (USE)
# │   │   └── Output: 512-d
# │   └── txt_proj (Dense 256 + ReLU)
# │       └── Output: 256-d
# │
# ├── 🔗 Fusion
# │   ├── Concatenate (img_proj + txt_proj) → 512-d
# │   └── fusion_head
# │       ├── Dense(128)
# │       └── L2 Normalize
# │           └── Output: 128-d embedding (final joint embedding)


In [132]:
#load pretrained model

resnet = tf.keras.applications.ResNet50(include_top=False, pooling='avg', input_shape=(224, 224, 3))
resnet.trainable = False


#for text
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


In [133]:
#Model

#adding our own layers dense layers at these models
img_proj = tf.keras.Sequential([   #a small neural network over our resnet , its weight we will train to get best embeddings out
    layers.Dense(256,activation = 'relu')
])

text_proj = tf.keras.Sequential([
    layers.Dense(256, activation='relu')
])

fusion_head = tf.keras.Sequential([  #also a layer to train in which we pass concat(txt,img) emb  and get final emb  
    layers.Dense(128), 
    layers.Lambda(lambda x: tf.math.l2_normalize(x,axis=1))   #this is my model to train
])


#3 layer to train --> img_proj  , text_proj , fusion_head


In [134]:
#Image Embedding

def get_image_embedding(image_path,model = resnet,projector = img_proj): #projector is out new layer we are adding
    img = tf.keras.preprocessing.image.load_img(image_path,target_size = (224,224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    img = tf.expand_dims(img,axis=0)   #add batch here as we pass image in a batch to resent
    features = model.predict(img)
    projected = projector(features)
    return projected.numpy()[0] # Remove batch, shape (256,)

    

In [135]:
#Text embedding

def get_text_embedding(caption, model=use, projector=text_proj):
    features = model([caption])  # shape: (1, 512)
    projected = projector(features)      # shape: (1, 256)
    return projected.numpy()[0]


In [136]:
def get_fused_embeddings(image_path,caption,model_img = resnet ,model_text=use,proj_img=img_proj, proj_txt=text_proj,fusion_model = fusion_head):
    img_emb = get_image_embedding(image_path,model_img,proj_img)  #see we send model also as can happen we change model but then we don't need to remove resent and ue , as from here we are controllin gmodel we are sending , not using by default inthe get_imaage_emb
    txt_emb = get_text_embedding(caption , model_text, proj_txt)

    fused = tf.concat([img_emb, txt_emb], axis=0)  # ✅ axis=0 for 1D arrays        
    output = fusion_model(tf.expand_dims(fused, axis=0))  # Add batch, then normalize
    return output.numpy()[0]  # shape: (128,)

#this is my model see get embeddings , fused them , but just didn't train like a,b  a,c distance

In [ ]:
# triplet_emb = []
# for anchor, pos,neg in triplets:
#     anchor_caption = image_captions[anchor][0]
#     pos_caption = image_captions[pos][0]
#     neg_caption = image_captions[neg][0]

#     anchor_path =   f"flickr8k/images/{anchor}"           #this is image path flickr8k/Images/3637013_c675de7705.jpg
#     pos_path = f"flickr8k/images/{pos}"
#     neg_path = f"flickr8k/images/{neg}"

#     anchor_emb = get_fused_embeddings(anchor_path, anchor_caption, resnet,use,img_proj,text_proj,fusion_head)
#     pos_emb = get_fused_embeddings(pos_path,pos_caption,resnet,use,img_proj,text_proj,fusion_head)
#     neg_emb =  get_fused_embeddings(neg_path,neg_caption,resnet,use,img_proj,text_proj,fusion_head)

#     triplet_emb.append((anchor_emb, pos_emb, neg_emb))


In [138]:
import pickle
pickle.dump(triplet_emb,open('triplet_emb.pkl','wb'))


In [139]:
# from tqdm import tqdm
# triplet_emb_nouns = []

# for anchor, pos, neg in tqdm(triplets):
#     # 1️⃣ Build “caption” from noun keywords instead of real sentence
#     anchor_caption = " ".join(image_keywords[anchor])
#     pos_caption    = " ".join(image_keywords[pos])
#     neg_caption    = " ".join(image_keywords[neg])

#     # 2️⃣ Build the image file paths
#     anchor_path = f"flickr8k/images/{anchor}"
#     pos_path    = f"flickr8k/images/{pos}"
#     neg_path    = f"flickr8k/images/{neg}"

#     # 3️⃣ Get fused embeddings using your existing function
#     anchor_emb = get_fused_embeddings(anchor_path, anchor_caption, resnet,use,img_proj,text_proj,fusion_head)
#     pos_emb    = get_fused_embeddings(pos_path, pos_caption, resnet,use,img_proj,text_proj,fusion_head)
#     neg_emb    = get_fused_embeddings(neg_path, neg_caption, resnet,use,img_proj,text_proj,fusion_head)

#     # 4️⃣ Append to new list
#     triplet_emb_nouns.append((anchor_emb, pos_emb, neg_emb))


In [140]:
pickle.dump(triplet_emb_nouns,open('triplet_emb_nouns.pkl','wb'))

In [141]:
#Load & Stack Precomputed Embeddings

import pickle
import numpy as np

# Load your pickled triplet embeddings
with open("triplet_emb.pkl", "rb") as f:
    triplet_emb = pickle.load(f)
    # triplet_emb: list of tuples (anchor_emb, pos_emb, neg_emb)

# Stack into three NumPy arrays of shape (N,128)
anchors   = np.stack([t[0] for t in triplet_emb])  # shape: (N,128)    #see here using stack as it is efficient in case of batch ooperations
positives = np.stack([t[1] for t in triplet_emb])  # shape: (N,128)
negatives = np.stack([t[2] for t in triplet_emb])  # shape: (N,128)


# | Feature              | `list`                    | `np.stack()`                  |
# | -------------------- | ------------------------- | ----------------------------- |
# | Type                 | Python `list` of arrays   | Single `numpy.ndarray`        |
# | Shape                | N objects of shape (128,) | One array of shape (N, 128)   |
# | Supports vector math | ❌ (must loop manually)    | ✅ Efficient & fast operations |
# | TensorFlow friendly  | ❌ Hard to use             | ✅ Preferred for training      |
# | Memory efficient     | ❌ More overhead           | ✅ Compact & optimized         |


In [143]:
print(len(anchors[0]))


128


In [ ]:
#Build the tf.data.Dataset   ,see pipeline belwo and explaination

import tensorflow as tf

# Generator yields one triplet of embeddings at a time
def triplet_generator():
    for a, p, n in zip(anchors, positives, negatives):
        yield a, p, n

#  What it is: A little Python function that, each time you ask it, gives you the next (anchor, positive, negative) triple of embeddings.
# Why: It’s a convenient way to stream your precomputed 128‑dim vectors one triplet at a time, without loading them all at once somewhere else.       

# Create Dataset with correct output types/shapes
ds = tf.data.Dataset.from_generator(
    triplet_generator,
    output_signature=(
        tf.TensorSpec((128,), tf.float32),   # Only when you call .batch(32) does TensorFlow accumulate 32 individual triplet elements into one batch.
        tf.TensorSpec((128,), tf.float32),
        tf.TensorSpec((128,), tf.float32),
    )
)



# What it does:
# Wraps your triplet_generator() in a TensorFlow object called a Dataset.
# Tells TensorFlow, “Each item I yield is three float32 vectors of length 128.”
# Why:
# This turns your plain‑Python generator into something TensorFlow can optimize and plug directly into model.fit().
# You get back a “stream” of (anchor, positive, negative) triplets as tf.Tensor objects, not NumPy arrays or Python lists.


#in other dataset(Static) (Python list / NumPy array / Pandas DataFrame) we first load whole data then fn , in case of large data RAM full

#in tf.data.Dataset(Streaming)     A tf.data.Dataset is TensorFlow’s way of representing and managing your data as a pipeline   A tf.data.Dataset is TensorFlow’s way of representing and managing your data as a pipeline of transformations and batches,(we creatae a pipeline , tell how to read the data we have ,  You define how to read, preprocess, and batch data.)  Data is fetched “just in time” in small pieces (streams) during training. ,  You never need to load the entire dataset into RAM.


# Shuffle, batch, and prefetch for performance
BATCH_SIZE = 32
ds = (
    ds
    .shuffle(buffer_size=len(anchors))
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)


# What it does:


# .shuffle(buffer_size=len(anchors)):->
# Keeps a “buffer” of that many triplets and randomly picks the next one from within the buffer.
# Why:
# Prevents the model from seeing the same sequence of triplets every epoch, which helps it generalize better.



# .batch(BATCH_SIZE):->
# Rather than returning one triplet at a time, it groups BATCH_SIZE triplets into a single “batch.”
# So instead of (anchor, pos, neg) shapes (128,), you now get three tensors of shape (BATCH_SIZE, 128).




# .prefetch(tf.data.AUTOTUNE):->
# While your model is busy training on one batch, TensorFlow prepares (loads, shuffles, batches) the next batch in the background.


# Composable Transformations
# With a Dataset you can chain simple operations:

# python
# Copy
# Edit
# ds = tf.data.Dataset.from_generator(...)
#    .shuffle(buffer_size=1000)
#    .map(preprocess_fn, num_parallel_calls=4)
#    .batch(32)
#    .prefetch(1)
# Each of these methods returns a new Dataset:

# shuffle: randomly reorders elements
# map: applies a function (e.g. image decoding, augmentation)
# batch: groups elements into fixed‑size batches
# prefetch: overlaps data preparation with model execution




